In [1]:
import os
import exifread  #pip install exifread

In [2]:
wp='J:/DS_refine/SQL-lyn/exifExtract/image/' #图片所在位置

### exifread使用

In [3]:
#根据官方教程尝试exifread  https://pypi.org/project/ExifRead/
with open(wp+'2049425142.jpg', 'rb') as f:
    exif_tag=exifread.process_file(f)

In [4]:
exif_tag

{'Image ImageWidth': (0x0100) Short=3120 @ 18,
 'Image ImageLength': (0x0101) Short=4160 @ 30,
 'Image BitsPerSample': (0x0102) Short=[8, 8, 8] @ 240,
 'Image ImageDescription': (0x010E) ASCII=smart @ 266,
 'Image Make': (0x010F) ASCII=HUAWEI @ 182,
 'Image Model': (0x0110) ASCII=BKK-AL10 @ 190,
 'Image XResolution': (0x011A) Ratio=72 @ 200,
 'Image YResolution': (0x011B) Ratio=72 @ 208,
 'Image ResolutionUnit': (0x0128) Short=Pixels/Inch @ 114,
 'Image Software': (0x0131) ASCII=BKK-AL10 8.2.0.187(C00) @ 216,
 'Image DateTime': (0x0132) ASCII=2019:08:23 20:55:32 @ 246,
 'Image YCbCrPositioning': (0x0213) Short=Centered @ 150,
 'Image ExifOffset': (0x8769) Long=272 @ 162,
 'GPS GPSVersionID': (0x0000) Byte=[2, 2, 0, 0] @ 900,
 'GPS GPSLatitudeRef': (0x0001) ASCII=N @ 912,
 'GPS GPSLatitude': (0x0002) Ratio=[39, 57, 48267/1250] @ 1028,
 'GPS GPSLongitudeRef': (0x0003) ASCII=E @ 936,
 'GPS GPSLongitude': (0x0004) Ratio=[116, 29, 10533/500] @ 1004,
 'GPS GPSAltitudeRef': (0x0005) Byte=1 @ 

In [6]:
exif_tag['GPS GPSLongitude']

(0x0004) Ratio=[116, 29, 10533/500] @ 1004

In [7]:
exif_tag['GPS GPSLongitude'].printable   #print(exif_tag['GPS GPSLongitude'])
#看坐标的特点，60进制，度，分，秒

'[116, 29, 10533/500]'

In [8]:
type(exif_tag['GPS GPSLongitude'])

exifread.classes.IfdTag

In [10]:
exif_tag['GPS GPSLongitude'],exif_tag['GPS GPSLatitude']

((0x0004) Ratio=[116, 29, 10533/500] @ 1004,
 (0x0002) Ratio=[39, 57, 48267/1250] @ 1028)

In [17]:
def extractExif(fpath):#提取坐标
    try:
        with open(fpath,'rb') as rf:
            exif=exifread.process_file(rf)
        eDate=exif['EXIF DateTimeOriginal'].printable
        eLon=exif['GPS GPSLongitude'].printable
        eLat=exif['GPS GPSLatitude'].printable
        lon=eLon[1:-1].replace(' ','').replace('/',',').split(',')
        #'[116, 29, 10533/500]' to [116,29,10533,500]  type==(list)
        lon=float(lon[0])+float(lon[1])/60+float(lon[2])/float(lon[3])/3600
        lat=eLat[1:-1].replace(' ','').replace('/',',').split(',')
        lat=float(lat[0])+float(lat[1])/60+float(lat[2])/float(lat[3])/3600
        return [lon,lat,eDate]  #经度,纬度,拍摄时间
    except Exception as e:
        print(e,fpath)
        return None

In [15]:
#测试
einfo=extractExif(wp+'2049425142.jpg')
einfo

(116.489185, 39.960726, '2019:08:23 20:55:32')

In [18]:
wpt='J:/DS_refine/SQL-lyn/exifExtract/image'
latLons=[]
for root, dirs, files in os.walk(wpt):
    print(len(files))
    for f in files:
        exif=extractExif('{0}/{1}'.format(wpt,f))
        if exif:
            exif[2]=exif[2]+' '+f
            latLons.append(exif)
        else:
            print(f,'exif is None')

37
'GPS GPSLongitude' J:/DS_refine/SQL-lyn/exifExtract/image/IMG_20151024_122254.jpg
IMG_20151024_122254.jpg exif is None
'GPS GPSLongitude' J:/DS_refine/SQL-lyn/exifExtract/image/IMG_20170904_121946.jpg
IMG_20170904_121946.jpg exif is None
'GPS GPSLongitude' J:/DS_refine/SQL-lyn/exifExtract/image/IMG_20160119_093407.jpg
IMG_20160119_093407.jpg exif is None
'GPS GPSLongitude' J:/DS_refine/SQL-lyn/exifExtract/image/IMG_20160119_101116.jpg
IMG_20160119_101116.jpg exif is None
'GPS GPSLongitude' J:/DS_refine/SQL-lyn/exifExtract/image/IMG_20170430_180820.jpg
IMG_20170430_180820.jpg exif is None
'GPS GPSLongitude' J:/DS_refine/SQL-lyn/exifExtract/image/IMG_20160826_090936.jpg
IMG_20160826_090936.jpg exif is None
'GPS GPSLongitude' J:/DS_refine/SQL-lyn/exifExtract/image/IMG_20160918_104806.jpg
IMG_20160918_104806.jpg exif is None
'GPS GPSLongitude' J:/DS_refine/SQL-lyn/exifExtract/image/IMG_20180704_143118.jpg
IMG_20180704_143118.jpg exif is None
'GPS GPSLongitude' J:/DS_refine/SQL-lyn/exifE

In [19]:
len(latLons)

27

In [21]:
latLons=[[i[1],i[0],i[2]] for i in latLons]

### 输出坐标用于地图可视化

#### marker-clustering

In [22]:
'var addressPoints = {0};'.format(str(latLons)) #marker-clustering-of-lyns-geopic 用

"var addressPoints = [[39.87754940972222, 116.45478844638889, '2019:05:21 12:26:24 1148888330.jpg'], [40.13971354, 116.46029043, '2019:04:21 09:53:07 1886044344.jpg'], [39.960726, 116.489185, '2019:08:23 20:55:32 2049425142.jpg'], [24.313702999722224, 109.406884, '2019:02:01 10:29:18 1747162316.jpg'], [39.13459399972222, 117.19196144972223, '2018:11:11 13:50:26 1809845184.jpg'], [39.91143417333333, 116.53287506083333, '2015:11:15 21:06:39 IMG_20151115_210639.jpg'], [40.43969345083333, 116.6190948486111, '2015:10:24 08:49:30 IMG_20151024_084930.jpg'], [40.044178008888885, 116.26774597166667, '2017:06:07 14:01:22 IMG_20170607_140122.jpg'], [40.020881652777774, 115.44576263416667, '2017:09:04 13:44:48 IMG_20170904_134448.jpg'], [24.248270999722223, 109.43681299972222, '2016:01:19 09:23:15 IMG_20160119_092315.jpg'], [32.6520327825, 116.26906168444444, '2017:04:30 17:59:20 IMG_20170430_175920.jpg'], [32.646699274166664, 116.267494965, '2017:04:30 18:12:09 IMG_20170430_181209.jpg'], [32.6451

#### 基于百度地图进行点坐标可视化

In [23]:
#坐标转换：可用 https://github.com/QLWeilcf/coordTransform_py
import coordTransform as cts

In [24]:
#测试效果
cts.wgs84_to_bd09(latLons[2][1],latLons[2][0]) #lon,lat

[116.50170875327169, 39.96807708061444]

In [25]:
bdLatLon=[cts.wgs84_to_bd09(i[1],i[0]) for i in latLons]

In [29]:
with open('./baiduMarkersjs.js','w+') as wf:
    for i in range(len(latLons)):
        wf.write('''
            var subwaynextc{0} = new BMap.Marker(new BMap.Point({1},{2}),{3}title:"{4}" {5});// 创建标注 
            map.addOverlay(subwaynextc{0});
            
            subwaynextc{0}.addEventListener("click", function(){3}//监听标注事件   
                var infoWindow = new BMap.InfoWindow("cd<br>{4} <br>",{3}    
                     width : 250, height: 160, title : "{4}"{5}); // 创建信息窗口对象
                map.openInfoWindow(infoWindow, new BMap.Point({1},{2}));  // 打开信息窗口  
           {5});
            '''.format(i+1,bdLatLon[i][0],bdLatLon[i][1],'{',latLons[i][2],'}'))

### 制作地图故事

In [34]:
#选几个典型坐标生成storymap-markers.js
slng=[latLons[17],latLons[4],latLons[12],latLons[24],latLons[3]]
scity=['beijing','tianjin','fuyang','ningbo','liuzhou']

In [38]:
for i in range(len(slng)):
    print('\t %s: {lat:%f, lon: %f, zoom: 8},'%(scity[i],slng[i][0],slng[i][1]))

beijing: {lat:39.886426, lon: 116.404762, zoom: 8},
tianjin: {lat:39.134594, lon: 117.191961, zoom: 8},
fuyang: {lat:32.645140, lon: 116.268333, zoom: 8},
ningbo: {lat:29.763531, lon: 121.898233, zoom: 8},
liuzhou: {lat:24.313703, lon: 109.406884, zoom: 8},


In [36]:
slng

[[39.886426,
  116.40476199972223,
  '2016:10:05 15:31:03 IMG_20161005_153103.jpg'],
 [39.13459399972222, 117.19196144972223, '2018:11:11 13:50:26 1809845184.jpg'],
 [32.645139996944444,
  116.26833340055555,
  '2017:04:30 18:17:56 IMG_20170430_181756.jpg'],
 [29.76353099972222,
  121.898233,
  '2018:07:06 09:27:03 IMG_20180706_092703.jpg'],
 [24.313702999722224, 109.406884, '2019:02:01 10:29:18 1747162316.jpg']]

In [60]:
latLon

[[116.45478844638889, 39.87754940972222, '1148888330.jpg'],
 [116.46029043, 40.13971354, '1886044344.jpg'],
 [116.489185, 39.960726, '2049425142.jpg'],
 [109.406884, 24.313702999722224, '1747162316.jpg'],
 [117.19196144972223, 39.13459399972222, '1809845184.jpg']]

### 参考资料

- [ExifRead](https://pypi.org/project/ExifRead/) ：便捷读取Exif的Python库
- [Leaflet](https://leafletjs.com/)：便捷友好的交互地图开源js库
- [markercluster.js](https://github.com/Leaflet/Leaflet.markercluster/)：地图标记点聚类库,Leaflet插件
- [storymap.js](https://github.com/atlefren/storymap)：地图上的故事,Leaflet插件
